In [1]:
"""This Python module is an implementation of the SHA-256 algorithm.
From https://github.com/keanemind/Python-SHA-256"""

import time

K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]

def generate_hash(message: bytearray) -> bytearray:
    """Return a SHA-256 hash from the message passed.
    The argument should be a bytes, bytearray, or
    string object."""

    # Begin start time
    start_time = time.time()
    
    if isinstance(message, str):
        message = bytearray(message, 'ascii')
    elif isinstance(message, bytes):
        message = bytearray(message)
    elif not isinstance(message, bytearray):
        raise TypeError

    # Padding
    length = len(message) * 8 # len(message) is number of BYTES!!!
    message.append(0x80)
    while (len(message) * 8 + 64) % 512 != 0:
        message.append(0x00)

    message += length.to_bytes(8, 'big') # pad to 8 bytes or 64 bits

    assert (len(message) * 8) % 512 == 0, "Padding did not complete properly!"

    # Parsing
    blocks = [] # contains 512-bit chunks of message
    for i in range(0, len(message), 64): # 64 bytes is 512 bits
        blocks.append(message[i:i+64])

    # Setting Initial Hash Value
    h0 = 0x6a09e667
    h1 = 0xbb67ae85
    h2 = 0x3c6ef372
    h3 = 0xa54ff53a
    h5 = 0x9b05688c
    h4 = 0x510e527f
    h6 = 0x1f83d9ab
    h7 = 0x5be0cd19

    # SHA-256 Hash Computation
    for message_block in blocks:
        # Prepare message schedule
        message_schedule = []
        for t in range(0, 64):
            if t <= 15:
                # adds the t'th 32 bit word of the block,
                # starting from leftmost word
                # 4 bytes at a time
                message_schedule.append(bytes(message_block[t*4:(t*4)+4]))
            else:
                term1 = _sigma1(int.from_bytes(message_schedule[t-2], 'big'))
                term2 = int.from_bytes(message_schedule[t-7], 'big')
                term3 = _sigma0(int.from_bytes(message_schedule[t-15], 'big'))
                term4 = int.from_bytes(message_schedule[t-16], 'big')

                # append a 4-byte byte object
                schedule = ((term1 + term2 + term3 + term4) % 2**32).to_bytes(4, 'big')
                message_schedule.append(schedule)

        assert len(message_schedule) == 64

        # Initialize working variables
        a = h0
        b = h1
        c = h2
        d = h3
        e = h4
        f = h5
        g = h6
        h = h7

        # Iterate for t=0 to 63
        for t in range(64):
            t1 = ((h + _capsigma1(e) + _ch(e, f, g) + K[t] +
                   int.from_bytes(message_schedule[t], 'big')) % 2**32)

            t2 = (_capsigma0(a) + _maj(a, b, c)) % 2**32

            h = g
            g = f
            f = e
            e = (d + t1) % 2**32
            d = c
            c = b
            b = a
            a = (t1 + t2) % 2**32

        # Compute intermediate hash value
        h0 = (h0 + a) % 2**32
        h1 = (h1 + b) % 2**32
        h2 = (h2 + c) % 2**32
        h3 = (h3 + d) % 2**32
        h4 = (h4 + e) % 2**32
        h5 = (h5 + f) % 2**32
        h6 = (h6 + g) % 2**32
        h7 = (h7 + h) % 2**32
        
    # Store end time and calculate total
    end_time = time.time()
    print("Software Execution time: ", end_time - start_time)
    return ((h0).to_bytes(4, 'big') + (h1).to_bytes(4, 'big') +
            (h2).to_bytes(4, 'big') + (h3).to_bytes(4, 'big') +
            (h4).to_bytes(4, 'big') + (h5).to_bytes(4, 'big') +
            (h6).to_bytes(4, 'big') + (h7).to_bytes(4, 'big'))

def _sigma0(num: int):
    """As defined in the specification."""
    num = (_rotate_right(num, 7) ^
           _rotate_right(num, 18) ^
           (num >> 3))
    return num

def _sigma1(num: int):
    """As defined in the specification."""
    num = (_rotate_right(num, 17) ^
           _rotate_right(num, 19) ^
           (num >> 10))
    return num

def _capsigma0(num: int):
    """As defined in the specification."""
    num = (_rotate_right(num, 2) ^
           _rotate_right(num, 13) ^
           _rotate_right(num, 22))
    return num

def _capsigma1(num: int):
    """As defined in the specification."""
    num = (_rotate_right(num, 6) ^
           _rotate_right(num, 11) ^
           _rotate_right(num, 25))
    return num

def _ch(x: int, y: int, z: int):
    """As defined in the specification."""
    return (x & y) ^ (~x & z)

def _maj(x: int, y: int, z: int):
    """As defined in the specification."""
    return (x & y) ^ (x & z) ^ (y & z)

def _rotate_right(num: int, shift: int, size: int = 32):
    """Rotate an integer right."""
    return (num >> shift) | (num << size - shift)

In [2]:
print(generate_hash("abc123").hex())

Software Execution time:  0.008591413497924805
6ca13d52ca70c883e0f0bb101e425a89e8624de51db2d2392593af6a84118090


In [3]:
# Write to a hash file on the board, this will be used later. The idea is
# if a pass phrase hash matches one of the hashes in this file, the user will
# get a secret message
f = open("hash.txt", "w")
f.write(generate_hash("apple").hex())
f.close()

#open and read the file after the appending:
f = open("hash.txt", "r")
print("The hash file now contains: ", f.read())

Software Execution time:  0.006258487701416016
The hash file now contains:  3a7bd3e2360a3d29eea436fcfb7e44c735d117c42d1c1835420b6b9942dd4f1b


In [4]:
def check_pass(inputpass):
    hashfile = open('hash.txt', 'r')
    hashlist = hashfile.readlines()
    hashfile.close()
    print("hashlist: ", hashlist)
    found = False
    for _hash in hashlist:
        if _hash == generate_hash(inputpass).hex():
            print ("Correct password! The secret message is: I love mangoes!")
            found = True

    if not found:
        print("WRONG!")

In [5]:
check_pass("mango")
print("*****************************************************")
check_pass("apple")

hashlist:  ['3a7bd3e2360a3d29eea436fcfb7e44c735d117c42d1c1835420b6b9942dd4f1b']
Software Execution time:  0.007336139678955078
WRONG!
*****************************************************
hashlist:  ['3a7bd3e2360a3d29eea436fcfb7e44c735d117c42d1c1835420b6b9942dd4f1b']
Software Execution time:  0.008130788803100586
Correct password! The secret message is: I love mangoes!


In [6]:
# We want the input to the sha256 hardware function to be a stream of integers
# representing the input string in ASCII. IT MUST BE 64 ELEMENTS LONG WITH THE
# 64TH ELEMENT BEING 0x18
initArray = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0x18]
def str2ascii(string, nparray):
    index = 0
    insert = []
    insert.extend(ord(element) for element in string)
    for i in insert:
        nparray[index] = insert[index]
        index += 1
    print(nparray)
    
def output2hash(outstream):
    finaloutput = ""
    for i in outstream:
        finaloutput += ord(i)
    return finaloutput

In [7]:
str2ascii("abc", initArray)

[97, 98, 99, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24]


In [8]:
# Import the overlay
from pynq import Overlay
overlay = Overlay('/home/xilinx/jupyter_notebooks/mynotebooks/sha256/sha256.bit')
overlay.sha256_0.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0)
}

In [22]:
# Adding SHA256 Axi Stream Overlay
# NOTE: INPUT STRING MUST BE 63 CHARs or less
from pynq import allocate
import numpy as np
import timeit as t
from pynq import Overlay

def sha256_hardware(string):
    data_size = 64
    initArray = [0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,
                 0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,
                 0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,
                 0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,
                 0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00]
    str2ascii(string,initArray)
    overlay = Overlay('/home/xilinx/jupyter_notebooks/mynotebooks/sha256/sha256.bit')
    dma = overlay.axi_dma
    dma_send = overlay.axi_dma.sendchannel
    dma_recv = overlay.axi_dma.recvchannel
    hls_ip = overlay.sha256_0
    CONTROL_REGISTER = 0x00
    hls_ip.write(CONTROL_REGISTER, 0x01) # 0x01 will set bit 0
    input_buffer = allocate(shape=(data_size,),dtype=np.uint32)
    for i in range(data_size):
        input_buffer[i] = initArray[i]
    print("input buffer = ", input_buffer)
    start = t.default_timer()
    dma_send.transfer(input_buffer)
    output_buffer = allocate(shape=(32,), dtype=np.ubyte)
    dma_recv.transfer(output_buffer)
    end = t.default_timer()
    print("Axi_stream output list: ", output_buffer)
    print("Output hash:")
    for i in range(len(output_buffer)):
        print(format(output_buffer[i], '02x'), end='')
    print("")
    print("Hardware execution time: ", end - start)
    del input_buffer, output_buffer

In [23]:
sha256_hardware("abc123")

[97, 98, 99, 49, 50, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
input buffer =  [97 98 99 49 50 51  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
Axi_stream output list:  [245  16  48 228 165  98 253  34  88  62  61  18 227  61  34 243 141 174
 136 110 209  29 100 143 108  24 175 120  14  26 216 189]
Output hash:
f51030e4a562fd22583e3d12e33d22f38dae886ed11d648f6c18af780e1ad8bd
Hardware execution time:  0.0034696920001806575


In [24]:
sha256_hardware("apple")

[97, 112, 112, 108, 101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
input buffer =  [ 97 112 112 108 101   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
Axi_stream output list:  [106   1  58  79 248 212 161 210 179 129 186 215 200   5 236  35 254  70
 173  58  62 178 184 140  31 252  71 241 152 253   6  90]
Output hash:
6a013a4ff8d4a1d2b381bad7c805ec23fe46ad3a3eb2b88c1ffc47f198fd065a
Hardware execution time:  0.0018460250000771339


In [26]:
print("HERE IS A DEMONSTRATION OF THE SOFTWARE AND HARDWARE FUNCTIONS RUNNING WITH THE SAME INPUTS. PLEASE NOTE, THE RESULTS ARE DIFFERENT DUE TO DIFFERENCE IN CASH CALCULATION MATH AND INITIAL HASH SET IN THE PRELOADED REGISTERS. HOWEVER, THESE ARE STILL VALID HASHES")
print(generate_hash("abc").hex())
print("********************************************")
print(generate_hash("abc123").hex())
print("********************************************")
print(generate_hash("mango").hex())
print("********************************************")
print(generate_hash("apple").hex())
print("******************************************************************************************************************")
sha256_hardware("abc")
print("********************************************")
sha256_hardware("abc123")
print("********************************************")
sha256_hardware("mango")
print("********************************************")
sha256_hardware("apple")
print("********************************************")

HERE IS A DEMONSTRATION OF THE SOFTWARE AND HARDWARE FUNCTIONS RUNNING WITH THE SAME INPUTS. PLEASE NOTE, THE RESULTS ARE DIFFERENT DUE TO DIFFERENCE IN CASH CALCULATION MATH AND INITIAL HASH SET IN THE PRELOADED REGISTERS. HOWEVER, THESE ARE STILL VALID HASHES
Software Execution time:  0.005578041076660156
ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad
********************************************
Software Execution time:  0.0056040287017822266
6ca13d52ca70c883e0f0bb101e425a89e8624de51db2d2392593af6a84118090
********************************************
Software Execution time:  0.005628824234008789
6815f3c300383519de8e437497e2c3e97852fe8d717a5419d5aafb00cb43c494
********************************************
Software Execution time:  0.005520343780517578
3a7bd3e2360a3d29eea436fcfb7e44c735d117c42d1c1835420b6b9942dd4f1b
******************************************************************************************************************
[97, 98, 99, 0, 0, 0, 0, 0, 0, 0, 0, 0,